In [1]:
import os
import re
from pathlib import Path
from PyPDF2 import PdfReader
from market_agents.memory.config import load_config_from_yaml
from market_agents.memory.setup_db import DatabaseConnection
from market_agents.memory.embedding import MemoryEmbedder
from market_agents.memory.knowledge_base import MarketKnowledgeBase
from market_agents.memory.knowledge_base_agent import KnowledgeBaseAgent
from market_agents.memory.vector_search import MemoryRetriever
from market_agents.agents.market_agent import MarketAgent
from market_agents.inference.message_models import LLMConfig
from market_agents.environments.mechanisms.chat import ChatEnvironment


In [2]:
# Load config
config_path = Path("../market_agents/memory/memory_config.yaml")
memory_config = load_config_from_yaml(str(config_path))

# Initialize database connection and embedding service
db_conn = DatabaseConnection(memory_config)
embedder = MemoryEmbedder(memory_config)

# Initialize knowledge base with "shakespeare" prefix
shakespeare_kb = MarketKnowledgeBase(
    config=memory_config,
    db_conn=db_conn,
    embedding_service=embedder,
    table_prefix="shakespeare_kb"
)

In [3]:
def document_reader(file_path: str) -> str:
    """
    Read content from PDF or TXT files and return cleaned text
    
    Args:
        file_path (str): Path to the document file
        
    Returns:
        str: Cleaned text content from the document
    """
    text_content = ""
    
    if file_path.endswith('.pdf'):
        pdf_reader = PdfReader(file_path)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            cleaned_text = re.sub(r'FTLN \d+', '', page_text)
            cleaned_text = re.sub(r'^\d+\s*', '', cleaned_text, flags=re.MULTILINE)
            text_content += cleaned_text
    
    elif file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            text_content = f.read()
    else:
        raise ValueError("Unsupported file format. Please provide a PDF or TXT file.")
            
    return text_content

# Read document content
#file_path = "/Users/interstellarninja/Documents/projects/MarketAgents/hamlet_PDF_FolgerShakespeare.pdf"
file_path = "/Users/interstellarninja/Documents/projects/MarketAgents/examples/the_complete_works_of_william_shakespear.txt"
text_content = document_reader(file_path)

In [4]:
# Store in knowledge base with metadata
knowledge_id = shakespeare_kb.ingest_knowledge(
    text_content,
    metadata={
        "source": "The Complete Works of William Shakespeare",
        "type": "play",
        "author": "William Shakespeare"
    }
)
print(f"Successfully created Hamlet knowledge base with ID: {knowledge_id}")

Successfully created Hamlet knowledge base with ID: af5ef830-8830-453a-a03a-8792ef9ce17e


In [5]:
# Initialize retriever
retriever = MemoryRetriever(memory_config, db_conn, embedder)

# Create knowledge base agent
kb_agent = KnowledgeBaseAgent(
    market_kb=shakespeare_kb,
    retriever=retriever
)

In [9]:
# Test some queries
def query_hamlet(question: str):
    print(f"\nQuestion: {question}")
    print("-" * 80)
    
    results = kb_agent.retrieve(question, "shakespeare_kb")
    
    for i, chunk in enumerate(results, 1):
        print(f"\nResult {i} (Similarity: {chunk.similarity:.3f})")
        print("Text:", chunk.text)
        print("\nContext:", chunk.context)
        print("-" * 80)

# Try different queries
queries = [
    "What is Hamlet's famous 'To be or not to be' soliloquy about?",
    "How does Ophelia die in the play?",
    "What is the role of the ghost in the play?",
    "Describe the relationship between Hamlet and Horatio"
]

for query in queries:
    query_hamlet(query)


Question: What is Hamlet's famous 'To be or not to be' soliloquy about?
--------------------------------------------------------------------------------

Result 1 (Similarity: 0.764)
Text: A lower place, note well,
May make too great an act. For learn this, Silius:
Better to leave undone than by our deed
Acquire too high a fame when him we serve’s away. Caesar and Antony have ever won
More in their officer, than person. Sossius,
One of my place in Syria, his lieutenant,
For quick accumulation of renown,
Which he achieved by th’ minute, lost his favour. Who does i’ th’ wars more than his captain can
Becomes his captain’s captain; and ambition,
The soldier’s virtue, rather makes choice of loss
Than gain which darkens him.

Context: ...t, and some wine! A health to Lepidus!

LEPIDUS.
I am not so well as I should be, but I’ll ne’er out.

ENOBARBUS.
Not till you have slept. I fear me you’ll be in till then.

LEPIDUS.
Nay, certainly, I have heard the Ptolemies’ pyramises are very goodly
thi

In [8]:
import json


chat_env = ChatEnvironment(name="shakespeare_chat")

shakespeare_agent = MarketAgent.create(
    memory_config=memory_config,
    db_conn=db_conn,
    agent_id="shakespeare_agent",
    use_llm=True,
    llm_config=LLMConfig(
        client="openai",
        model="gpt-4o",
        temperature=0.5,
        max_tokens=1024
    ),
    knowledge_agent=kb_agent, 
    environments={"chat": chat_env},
)

# Add a test message to the chat
shakespeare_agent.task = "What is Hamlet's famous 'To be or not to be' soliloquy about?"
chat_env.mechanism.add_user_message(shakespeare_agent.task)

# Generate a response
perception = await shakespeare_agent.perceive(environment_name="chat")
print(json.dumps(perception, indent=4))

action = await shakespeare_agent.generate_action(environment_name="chat", perception=perception)
print(json.dumps(action, indent=4))

reflection = await shakespeare_agent.reflect(environment_name="chat")
print(json.dumps(reflection, indent=4))



Cognitive Memory Results:
Memory 1:
{'cognitive_step': 'perception', 'content': '{"monologue": "The user is inquiring about the famous soliloquy from Hamlet, which is a deeply philosophical reflection on existence and the nature of life and death. However, the retrieved documents seem to be excerpts from another Shakespearean play, \'Antony and Cleopatra,\' which do not directly relate to the user\'s question. This could create some confusion, but I can focus on addressing the user\'s query by providing insights into Hamlet\'s soliloquy.", "key_observations": ["User asked about Hamlet\'s \'To be or not to be\' soliloquy.", "Retrieved documents are from \'Antony and Cleopatra,\' not directly related to the user\'s question."], "strategy": ["Provide a summary of Hamlet\'s soliloquy focusing on its themes of existence, life, and death.", "Clarify any confusion by stating that the retrieved documents are not relevant to the user\'s question."], "confidence": 0.85}'}

Memory 2:
{'cognitive

In [14]:
shakespeare_agent.interactions

[{'id': 'shakespeare_agent',
  'name': 'agent',
  'system': {'role': 'system',
   'content': 'Role: You are a agent agent. You are a long-horizon task execution agent based on verbal reinforcement learning. The current date and time is 2025-01-22 11:41:51.\n\n'},
  'task': 'Tasks: Your are assigned with following tasks:\nPerceive the current state of the chat environment:\n\n## Environment State\n-   content: What is Hamlet\'s famous \'To be or not to be\' soliloquy about?\n  timestamp: 2025-01-22 11:41:49\n  role: user\n\n## Recent Cognitive Memories\n-   cognitive_step: perception\n  content: {"monologue": "The user is inquiring about the famous soliloquy from Hamlet, which is a deeply philosophical reflection on existence and the nature of life and death. However, the retrieved documents seem to be excerpts from another Shakespearean play, \'Antony and Cleopatra,\' which do not directly relate to the user\'s question. This could create some confusion, but I can focus on addressing t

In [16]:
shakespeare_agent.environments

{'chat': ChatEnvironment(name='shakespeare_chat', address=None, current_step=0, max_steps=10, action_space=ChatActionSpace(allowed_actions=[<class 'market_agents.environments.mechanisms.chat.ChatAction'>]), observation_space=ChatObservationSpace(allowed_observations=[<class 'market_agents.environments.mechanisms.chat.ChatObservation'>]), history=EnvironmentHistory(steps=[]), mechanism=ChatMechanism(sequential=True, chat_history=[ChatMessage(content="What is Hamlet's famous 'To be or not to be' soliloquy about?", timestamp='2025-01-22 11:41:49', role='user')]))}

In [18]:
shakespeare_agent.knowledge_agent

KnowledgeBaseAgent(market_kb=<market_agents.memory.knowledge_base.MarketKnowledgeBase object at 0x113ffbef0>, retriever=<market_agents.memory.vector_search.MemoryRetriever object at 0x117327a40>, knowledge_bases={}, retrieved_knowledge=[])

In [20]:
shakespeare_agent.last_perception

{'monologue': "The user has asked about Hamlet's 'To be or not to be' soliloquy, a classic reflection on existential themes. However, the documents retrieved are from 'Antony and Cleopatra,' which are unrelated to the user's query. This mismatch needs clarification to avoid confusion. My previous response addressed this by summarizing Hamlet's soliloquy and noting the irrelevance of the retrieved texts. I feel confident in my ability to provide insightful explanations while managing document discrepancies. 😊📚",
 'key_observations': ["User asked about Hamlet's 'To be or not to be' soliloquy.",
  "Retrieved documents are from 'Antony and Cleopatra,' not directly related to the user's question."],
 'strategy': ["Provide a summary of Hamlet's soliloquy focusing on its themes of existence, life, and death.",
  "Clarify any confusion by stating that the retrieved documents are not relevant to the user's question."],
 'confidence': 0.9}

In [21]:
shakespeare_agent.last_action

{'agent_id': 'agent_001',
 'action': {'thoughts': [{'reasoning': "The user asked about Hamlet's 'To be or not to be' soliloquy. I need to provide a summary of this soliloquy and clarify any document mismatches to ensure understanding."},
   {'reasoning': "The retrieved documents are from 'Antony and Cleopatra,' which is not related to the user's query. It's important to acknowledge this discrepancy to maintain clarity."},
   {'reasoning': 'I will summarize the soliloquy by focusing on its central themes of existence, life, and death, as these are the core aspects the user is likely interested in.'}],
  'final_answer': "Hamlet's 'To be or not to be' soliloquy is a profound meditation on the nature of existence and the human condition. In this soliloquy, Hamlet contemplates the pain and unfairness of life versus the fear of the uncertainty of death and what comes after. This internal debate reflects his existential crisis and the universal struggle with the meaning of life. Please note t

In [27]:
# Generate a response
perception_prompt = await shakespeare_agent.perceive(environment_name="chat", return_prompt=True)
print(json.dumps(perception_prompt.model_dump_json(), indent=4))


Cognitive Memory Results:
Memory 1:
{'cognitive_step': 'action', 'content': '{"agent_id": "Agent007", "action": {"thoughts": [{"reasoning": "The user asked about the famous \'To be or not to be\' soliloquy from Hamlet, which is a central piece of literature discussing themes of existence, life, and death. It seems the retrieved documents are not related to this query, as they are excerpts from \'Antony and Cleopatra.\' I need to clarify this to the user and provide an accurate response about Hamlet\'s soliloquy."}, {"reasoning": "To address the user\'s question, I should provide a brief summary of the soliloquy, highlighting its main themes and the philosophical questions it raises. This will help the user understand the significance of the soliloquy in the context of the play."}, {"reasoning": "I will also inform the user that the retrieved documents are not directly related to their question, ensuring that any potential confusion is minimized."}], "final_answer": "Hamlet\'s famous \

/Users/interstellarninja/miniconda3/envs/market-agents/lib/python3.12/site-packages/pydantic/main.py:415: UserWarning: Pydantic serializer warnings:
  Expected `Union[str, generator]` but got `list` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_json(


In [32]:
print(perception_prompt.new_message)

Tasks: Your are assigned with following tasks:
Perceive the current state of the chat environment:

## Environment State
-   content: What is Hamlet's famous 'To be or not to be' soliloquy about?
  timestamp: 2025-01-22 11:41:49
  role: user

## Recent Cognitive Memories
-   cognitive_step: action
  content: {"agent_id": "Agent007", "action": {"thoughts": [{"reasoning": "The user asked about the famous 'To be or not to be' soliloquy from Hamlet, which is a central piece of literature discussing themes of existence, life, and death. It seems the retrieved documents are not related to this query, as they are excerpts from 'Antony and Cleopatra.' I need to clarify this to the user and provide an accurate response about Hamlet's soliloquy."}, {"reasoning": "To address the user's question, I should provide a brief summary of the soliloquy, highlighting its main themes and the philosophical questions it raises. This will help the user understand the significance of the soliloquy in the conte